In [ ]:
import numpy as np
import pandas as pd
from utils.utils import PROJECT_DATA_DIR
import os
import mxnet as mx
import tarfile
import warnings
warnings.filterwarnings("ignore")

C:\Users\gilbe\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
karm  = pd.read_csv(os.path.join(PROJECT_DATA_DIR, 'all_training_400_karm.csv'), sep='\s+', header=None)

In [ ]:
karm.head()

In [ ]:
print(karm.shape)
print('labels:', karm[karm.columns[-1]].unique())

In [ ]:
data = pd.read_csv(os.path.join(PROJECT_DATA_DIR, 'all_training_200_4.csv'), sep='\s+', header=None)

In [ ]:
df = pd.read_csv(os.path.join(PROJECT_DATA_DIR, 'all_training_100.csv'),sep='\s+', header=None)

In [ ]:
df.head()

In [ ]:
print(df.shape)
print('labels:', df[df.columns[-1]].unique())


In [ ]:
print(data[data.columns[-1]].unique())
data.head()

In [ ]:
data.shape

In [ ]:
df1 = pd.read_csv(os.path.join(PROJECT_DATA_DIR, 'all_test_2000_1.csv'),sep='\s+', header=None)

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
#data_iter = mx.io.CSVIter(
#    data_csv=os.path.join(PROJECT_DATA_DIR, 'all_test_2000_1.csv'),data_shape=(1,), batch_size=2**9)

In [ ]:
#data_iter

In [ ]:
#for batch in data_iter:
#    print([batch.data, batch.pad])

In [ ]:
# Work with neural networks

In [ ]:
sensor_train = pd.read_csv(os.path.join(PROJECT_DATA_DIR, 'all_training_400_minisensor_1.csv'), sep='\s+', header=None)
sensor_test = pd.read_csv(os.path.join(PROJECT_DATA_DIR, 'all_test_400_minisensor.csv'), sep='\s+', header=None)

In [ ]:
sensor_train.head()

In [ ]:
sensor_test.head()

In [ ]:
sensor_train[sensor_train.columns[-1]].unique()

In [ ]:
sensor_train[sensor_train.columns[-1]].unique()

In [ ]:
sensor_test[sensor_test.columns[-1]].unique()

In [ ]:
from sklearn.preprocessing import (MinMaxScaler,
                                   StandardScaler,
                                   Imputer,
                                   QuantileTransformer)

In [ ]:
ytrain = sensor_train[sensor_train.columns[-1]]
#ytrain.head()
xtrain = sensor_train.drop(sensor_train.columns[-1], axis=1) 

In [ ]:
xtrain.head()

In [ ]:
ytest = sensor_test[sensor_test.columns[-1]]
xtest = sensor_test.drop(sensor_test.columns[-1], axis=1) 

In [ ]:
ytrain_sc = ytrain.map(lambda x: 1 if (x == 2 or x == 3) else 0)
ytest_sc = ytest.map(lambda x: 1 if (x == 2 or x == 3) else 0)

In [ ]:
print(ytrain_sc.unique())
ytrain_sc.head()


## New data

In [ ]:
list_ = []
train_list = ['xab.csv', 'xaa.csv']
for x in train_list:
    df = pd.read_csv(os.path.join(PROJECT_DATA_DIR, x), sep='\s+', header=None)
    #    list_.append(df)
#train_xab = pd.concat(list_)

df = pd.read_csv(os.path.join(PROJECT_DATA_DIR, 'xab.csv'), sep='\s+', header=None)

In [ ]:
train_xab.head()

In [ ]:
train_xab.shape

In [ ]:
ytrain_xab = train_xab[train_xab.columns[-1]]
print(ytrain_xab.unique())


In [ ]:
xtrain_xab = train_xab.drop(train_xab.columns[-1], axis=1)

In [ ]:
xtrain_xab.head()

In [ ]:
test_xab = pd.read_csv(os.path.join(PROJECT_DATA_DIR, 'xab_test_400.csv'), sep='\s+', header=None)

In [ ]:
test_xab.head()

In [ ]:
test_xab.shape

In [ ]:
#xtest_xab.head()

In [ ]:
ytest_xab = test_xab[test_xab.columns[-1]]
print(ytest_xab.unique())
#xtest_xab = test_xab.drop(test_xab.columns[-1])

In [ ]:
xtest_xab = test_xab.drop(test_xab.columns[-1], axis=1)

In [ ]:
xtest_xab.shape

In [ ]:
xtest_xab.head()

In [ ]:
ytrain_xab = ytrain_xab.map(lambda x: 1 if x in [12, 13, 17, 20] else 0)
ytest_xab = ytest_xab.map(lambda x: 1 if x in [12, 13, 17, 20] else 0)

In [ ]:
print(ytrain_xab.unique())
ytrain_xab.head()

## Preprocessing

In [ ]:
xtrain.info()

In [ ]:
print(xtrain.shape)
xtrain.describe()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
xtrain[xtrain.columns[0:10:3]].plot()
plt.show()

In [ ]:
xtrain[xtrain.columns[1100:1120:3]].plot()
plt.show()

In [ ]:
xtrain[xtrain.columns[500:510:3]].plot()
plt.show()

In [ ]:
ytrain_sc.hist()
plt.show()

In [ ]:
ytest_sc.hist()
plt.show()

## Scaling and transforming data

In [ ]:
scaler = StandardScaler()#QuantileTransformer(output_distribution='normal')
xtrain_sc = scaler.fit_transform(xtrain)
xtest_sc = scaler.transform(xtest)

In [ ]:
print(ytrain_sc.shape)
print(xtrain_sc.shape)
type(xtrain_sc)


In [ ]:
plt.plot(xtrain_sc[:,0:10:3])
plt.show()


In [ ]:
plt.plot(xtest_sc[:,0:10:3])
plt.show()

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
from keras import optimizers
from sklearn.metrics import f1_score

In [ ]:
def model(drop_rate, lr, units, decay, no_features, no_outputs):
    optim = optimizers.Adam(lr=lr,
                            beta_1=0.9,
                            beta_2=0.999,
                            decay=1e-6)
    model = Sequential()
    model.add(BatchNormalization(input_shape=(xtrain.shape[1], )))
    model.add(Dense(units, activation='relu', input_dim=1200))
    model.add(Dense(1, activation='softmax'))
    model.compile(loss='categorical_crossentropy', 
                  optim=optim, 
                  metrics=['accuracy'])
    
    return model

In [ ]:
del model
model = Sequential()
#model.add(BatchNormalization(input_dim=1200))
model.add(Dense(units=1200, activation='relu', input_dim=1200), )
model.add(Dropout(0.4))
model.add(Dense(units=500))
model.add(Dropout(0.4))
model.add(Dense(units=75, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=1, activation='sigmoid'))


In [ ]:
lr = 0.001
optim = optimizers.Adam(lr=lr,
                        beta_1=0.9,
                        beta_2=0.999,
                        decay=1e-6)
model.compile(loss='binary_crossentropy',
                        optimizer=optim,
                        metrics=['accuracy'])

In [ ]:
history = model.fit(xtrain_sc,
                    ytrain_sc,
                    epochs=20,
                    batch_size=2**9, 
                    validation_data=(xtest_sc, ytest_sc))

In [ ]:
pred_train = model.predict_classes(xtrain_sc)
pred_test = model.predict_classes(xtest_sc)
print(pred_test)
print('')
print('unique values train data set', np.unique(pred_train))
print('unique values test data set:', np.unique(pred_test))
print('')
print('training score:', f1_score(ytrain_sc, pred_train))
print('test score:', f1_score(ytest_sc, pred_test))

In [ ]:
xtest_xab.shape

In [ ]:
type(xtest_xab)

In [ ]:
xtrain_xab.shape

In [ ]:
xtrain_xab.head()

In [ ]:
xtrain_xab.head()

In [ ]:
scaler_xab = QuantileTransformer(copy=False, output_distribution='normal', n_quantiles =100)
#k = int(0.5* len(xtest_xab))
#print(k)
#xtrain_ax = xtest_xab.iloc[0:k, :]
#xtest_ax = xtest_xab.iloc[k:, :]

#xtrain_ax = scaler_xab.fit_transform(xtrain_ax)
#xtest_ax = scaler_xab.transform(xtest_ax)

#xtrain_xab_sc = scaler_xab.fit_transform(xtest_xab)
#print(xtrain_xab_sc.shape)
#print(xtrain_ax.shape)
#print(xtest_ax.shape)
#xtest_xab_sc = scaler_xab.transform(xtest_ax)
xtrain_xab_sc = scaler_xab.fit_transform(xtrain_xab)
xtest_xab_sc = scaler_xab.transform(xtest_xab)

In [ ]:
#ytrain_ax = ytest_xab.iloc[:k]
#ytest_ax = ytest_xab.iloc[k:]

In [ ]:
ytrain_ax.shape

In [ ]:
ytest_ax.shape

In [ ]:
history = model.fit(xtrain_xab,
                    ytrain_xab,
                    epochs=20,
                    batch_size=2**9, 
                    validation_data=(xtest_xab, ytest_xab))

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(history.history(['loss']))
plt.plot(history.history(['val_loss']))
plt.xlabel('epoch')
#plt.legeng()
plt.show()

In [ ]:
drop_rate = 0.4
lr = 0.001
decay = 1e-6
units = 500
no_features = xtrain.shape[1]
no_outputs = 1
model = model(drop_rate, lr, decay, units, no_features, no_outputs)